In [ ]:
#pip install psycopg2-binary

In [39]:
#import sqlalchemy
import pandas as pd
import psycopg2
import psycopg2.extras as extras 
#from sqlalchemy import create_engine, inspect
from datetime import datetime

In [41]:
DROP_CREATE_DESTINATION_TABLE = False #Careful do not set to True!

In [43]:
#Get ETL_DATE for Incremental Ingestion (enabled if ETL_DATE set to 'CURRENT_DATE')
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('db_credentials.env')
load_dotenv(dotenv_path=dotenv_path)

ETL_DATE = os.getenv('ETL_DATE')

In [45]:
# For Incremental Ingestion (enabled if ETL_DATE set to 'CURRENT_DATE')
if ETL_DATE == 'CURRENT_DATE':
    ETL_DATE = datetime.today().strftime('%Y%m%d')
else:
   ETL_DATE = '20250322'
    
print(ETL_DATE)

20250328


In [49]:
df = pd.read_csv(f"data/silver_{ETL_DATE}_Airline_Reviews_LostLuggageLabel.csv")
print(len(df))
df.head()

52


,RowId,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Wifi & Connectivity,Value For Money,Recommended,Id,sentiment_label,sentiment_scores,review_gte_small_embeddings,lost_luggage_cosim,lost_luggage_kndist,is_lost_luggage_flag
0,0,Aer Lingus,2.0,“terrible customer service”,2025-03-26,False,Absolutely terrible customer service. Tried t...,NaN,NaN,Couple Leisure,...,3.0,1,no,661172569262a3a1c15d87e9459a0fefc8664914266518...,negative,0.948760,"[-0.006984285544604063, -0.012522410601377487,...",0.786482,0.653480,False
1,1,Air France,10.0,"""excellent customer service""",2025-03-28,True,My son was travelling on Kids Solo as an un...,NaN,NaN,Family Leisure,...,NaN,5,yes,925fcfda402034e92b439424f9eb35bec83154e497e641...,neutral,0.701676,"[-0.036641675978899, 0.022963620722293854, 0.0...",0.814622,0.608897,False
2,2,Air India,3.0,“it was not a pleasant experience”,2025-03-26,False,Recently travelled on Air India flight from T...,NaN,NaN,Family Leisure,...,1.0,1,no,c09a8175ab7eda10f0fddbbdc6227c44836d964901af2c...,negative,0.710657,"[0.0173887200653553, -0.020334232598543167, 0....",0.800439,0.631760,False
3,3,Air India Express,1.0,"""not accountable for any damage""",2025-03-23,True,No proper communication and very casual app...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,1.0,1,no,455e41f45426cc1ebd206caba0945f054613394cb208c5...,negative,0.902924,"[-0.009622704237699509, 0.012122679501771927, ...",0.890122,0.468782,True
4,4,AirAsia X,2.0,"""flight is delay for 20 hours""",2025-03-24,True,"Worse experience, my flight suppose to depa...",NaN,NaN,Business,...,NaN,1,no,a13ee7a23ddc22c11deadf130dbe3b23384427e2fd5e01...,negative,0.841998,"[0.025981714949011803, -0.01533507276326418, 0...",0.802163,0.629027,False


In [51]:
# Data Preprocessing
#needed_columns = [c for c in df_raw.columns if c != 'Unnamed: 0']
needed_columns = ['Id', 
 'RowId',
 'Airline Name',
 'Review_Title',
 'Review Date',
 'Verified',
 'Review',
 'Top Review Image Url',
 'Aircraft',
 'Type Of Traveller',
 'Seat Type',
 'Route',
 'Date Flown',
 'Seat Comfort',
 'Cabin Staff Service',
 'Food & Beverages',
 'Ground Service',
 'Inflight Entertainment',
 'Wifi & Connectivity',
 'Value For Money',
 'Recommended',
 'sentiment_label',
 'sentiment_scores',
 'Overall_Rating',
 'is_lost_luggage_flag',
 'lost_luggage_cosim']

In [53]:
df[needed_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Id                      52 non-null     object 
 1   RowId                   52 non-null     int64  
 2   Airline Name            52 non-null     object 
 3   Review_Title            52 non-null     object 
 4   Review Date             52 non-null     object 
 5   Verified                52 non-null     bool   
 6   Review                  52 non-null     object 
 7   Top Review Image Url    8 non-null      object 
 8   Aircraft                15 non-null     object 
 9   Type Of Traveller       52 non-null     object 
 10  Seat Type               52 non-null     object 
 11  Route                   52 non-null     object 
 12  Date Flown              52 non-null     object 
 13  Seat Comfort            50 non-null     float64
 14  Cabin Staff Service     49 non-null     floa

In [55]:
#Get DB credentials
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('db_credentials.env')
load_dotenv(dotenv_path=dotenv_path)

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

In [57]:
''' 
#Create bronze_airline_quality_reviews staging table
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    
    cursor = connection.cursor()
    
    cursor.execute("""
                DROP TABLE IF EXISTS "bronze_airline_quality_reviews";
                """)
            
    connection.commit()
    print(f"Table 'bronze_airline_quality_reviews' dropped successfully.")



    cursor = connection.cursor()
    
    # Create the users table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS bronze_airline_quality_reviews (
    	"Id" text NOT NULL, 
        "RowId" text NULL,
    	"Airline Name" text NOT NULL,
    	"Review_Title" text NOT NULL,
    	"Review Date" text NOT NULL,
    	"Verified" text NULL,
    	"Review" text NULL,
    	"Top Review Image Url" text NULL,
    	"Aircraft" text NULL,
    	"Type Of Traveller" text NULL,
    	"Seat Type" text NULL,
    	"Route" text NULL,
    	"Date Flown" text NULL,
    	"Seat Comfort" text NULL,
    	"Cabin Staff Service" text NULL,
    	"Food & Beverages" text NULL,
    	"Ground Service" text NULL,
    	"Inflight Entertainment" text NULL,
    	"Wifi & Connectivity" text NULL,
    	"Value For Money" text NULL,
    	"Recommended" text NULL,
    	sentiment_label text NULL,
        sentiment_scores text NULL,
    	"Overall_Rating" text NULL,
        is_lost_luggage_flag text NULL,
        lost_luggage_cosim text NULL
    );
    """)
    connection.commit()
    print("Table 'bronze_airline_quality_reviews' created successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error during drop-create operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
'''

' \n#Create bronze_airline_quality_reviews staging table\ntry:\n    # Connect to the PostgreSQL database\n    connection = psycopg2.connect(\n        dbname=DB_NAME,\n        user=DB_USER,\n        password=DB_PASSWORD,\n        host=DB_HOST,\n        port=DB_PORT\n    )\n    \n    cursor = connection.cursor()\n    \n    cursor.execute("""\n                DROP TABLE IF EXISTS "bronze_airline_quality_reviews";\n                """)\n            \n    connection.commit()\n    print(f"Table \'bronze_airline_quality_reviews\' dropped successfully.")\n\n\n\n    cursor = connection.cursor()\n    \n    # Create the users table\n    cursor.execute("""\n        CREATE TABLE IF NOT EXISTS bronze_airline_quality_reviews (\n    \t"Id" text NOT NULL, \n        "RowId" text NULL,\n    \t"Airline Name" text NOT NULL,\n    \t"Review_Title" text NOT NULL,\n    \t"Review Date" text NOT NULL,\n    \t"Verified" text NULL,\n    \t"Review" text NULL,\n    \t"Top Review Image Url" text NULL,\n    \t"Aircraf

In [59]:
#Insert dataframe to RDS using SQLAlchemy (We need to use psycopg2 for AWS Lambda instead of SQLAlchemy)
#conf ={
#    'host':DB_HOST,
#    'port':DB_PORT,
#    'database':DB_NAME,
#    'user':DB_USER,
#    'password':DB_PASSWORD
#}
#engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{user}".format(**conf))

#df.to_sql('bronze_airline_quality_reviews', engine, index=False, if_exists='replace')


#Insert dataframe to RDS using psycopg2 (We need to use psycopg2 for AWS Lambda instead of SQLAlchemy)
def insert_df_to_table(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()] 

    # Get column names from DataFrame
    create_cols = '" TEXT,"'.join(needed_columns)
    cols = '","'.join(needed_columns)
    #print(cols)

    # Create table if it does not exist
    create_query = f'DROP TABLE IF EXISTS {table}; CREATE TABLE IF NOT EXISTS {table} ("{create_cols}" TEXT);'
    print(create_query)
    create_cursor = conn.cursor()

    
    # Insert values to table
    query = f'INSERT INTO {table} ("{cols}") VALUES %s'
    print(query)
    cursor = conn.cursor()
    
    try: 
        create_cursor.execute(create_query)
        
        # Execute the SQL query with the data
        extras.execute_values(cursor, query, tuples)
        
        # Commit the transaction
        conn.commit()
        print("The dataframe is inserted")
        
    except (Exception, psycopg2.DatabaseError) as error: 
        # Handle any errors
        print(f"Error: {error}")
        conn.rollback()
        
    finally:
        create_cursor.close()
        cursor.close()
    
    
    
    
conn = psycopg2.connect( 
    database=DB_NAME, 
    user=DB_USER, 
    password=DB_PASSWORD, 
    host=DB_HOST, 
    port=DB_PORT
) 

insert_df_to_table(conn, df[needed_columns], 'bronze_airline_quality_reviews') 

DROP TABLE IF EXISTS bronze_airline_quality_reviews; CREATE TABLE IF NOT EXISTS bronze_airline_quality_reviews ("Id" TEXT,"RowId" TEXT,"Airline Name" TEXT,"Review_Title" TEXT,"Review Date" TEXT,"Verified" TEXT,"Review" TEXT,"Top Review Image Url" TEXT,"Aircraft" TEXT,"Type Of Traveller" TEXT,"Seat Type" TEXT,"Route" TEXT,"Date Flown" TEXT,"Seat Comfort" TEXT,"Cabin Staff Service" TEXT,"Food & Beverages" TEXT,"Ground Service" TEXT,"Inflight Entertainment" TEXT,"Wifi & Connectivity" TEXT,"Value For Money" TEXT,"Recommended" TEXT,"sentiment_label" TEXT,"sentiment_scores" TEXT,"Overall_Rating" TEXT,"is_lost_luggage_flag" TEXT,"lost_luggage_cosim" TEXT);
INSERT INTO bronze_airline_quality_reviews ("Id","RowId","Airline Name","Review_Title","Review Date","Verified","Review","Top Review Image Url","Aircraft","Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service","Food & Beverages","Ground Service","Inflight Entertainment","Wifi & Connectivity","Value For Mon

In [60]:

# Execute once on initial load
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

    if DROP_CREATE_DESTINATION_TABLE:
        cursor = connection.cursor()
        cursor.execute("""
                    DROP TABLE IF EXISTS "silver_airline_quality_reviews" CASCADE;
                    """)   
        connection.commit()
        print(f"Table 'silver_airline_quality_reviews' dropped successfully.")
    


    cursor = connection.cursor()
    
    # Create the users table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS silver_airline_quality_reviews (
    	"Id" varchar(64) NOT NULL PRIMARY KEY, 
        "RowId" int8 NULL,
    	"Airline Name" varchar(255) NOT NULL,
    	"Review_Title" varchar(1000) NOT NULL,
    	"Review Date" timestamp NOT NULL,
    	"Verified" bool NULL,
    	"Review" text NULL,
    	"Top Review Image Url" varchar(1000) NULL,
    	"Aircraft" varchar(255) NULL,
    	"Type Of Traveller" varchar(255) NULL,
    	"Seat Type" varchar(255) NULL,
    	"Route" varchar(1000) NULL,
    	"Date Flown" timestamp NULL,
    	"Seat Comfort" float8 NULL,
    	"Cabin Staff Service" float8 NULL,
    	"Food & Beverages" float8 NULL,
    	"Ground Service" float8 NULL,
    	"Inflight Entertainment" float8 NULL,
    	"Wifi & Connectivity" float8 NULL,
    	"Value For Money" float8 NULL,
    	"Recommended" varchar(3) NULL,
    	sentiment_label varchar(50) NULL,
        sentiment_scores float8 NULL,
    	"Overall_Rating" float8 NULL,
        is_lost_luggage_flag bool NULL,
        is_lost_luggage_score float8 NULL,
        created_by varchar(100) NOT NULL,
        created_date timestamp NOT NULL DEFAULT NOW(),
        lastmodified_by varchar(100) NOT NULL,
        lastmodified_date timestamp NOT NULL DEFAULT NOW()   
    );
    """)
    connection.commit()
    print("Table 'silver_airline_quality_reviews' created successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error during drop-create operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()


Table 'silver_airline_quality_reviews' created successfully.


In [61]:
# This checks if record already exists by Id before inserting
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

    cursor = connection.cursor()
    
    # Create the users table
    cursor.execute("""
        --TRUNCATE TABLE silver_airline_quality_reviews;
        INSERT INTO silver_airline_quality_reviews 
        (
        	"Id",
            "RowId",
        	"Airline Name",
        	"Review_Title",
        	"Review Date",
        	"Verified",
        	"Review",
        	"Top Review Image Url",
        	"Aircraft",
        	"Type Of Traveller",
        	"Seat Type",
        	"Route",
        	"Date Flown",
        	"Seat Comfort",
        	"Cabin Staff Service",
        	"Food & Beverages",
        	"Ground Service",
        	"Inflight Entertainment",
        	"Wifi & Connectivity",
        	"Value For Money",
        	"Recommended",
        	sentiment_label,
            sentiment_scores,
        	"Overall_Rating",
            is_lost_luggage_flag,
            is_lost_luggage_score,
            created_by,
            created_date,
            lastmodified_by,
            lastmodified_date           
        )
        SELECT
        	CAST("Id" AS varchar(64)),
            CAST("RowId" AS INT8),
        	"Airline Name",
        	"Review_Title",
        	CAST("Review Date" AS TIMESTAMP),
        	CAST("Verified" AS BOOL),
        	"Review",
        	"Top Review Image Url",
        	"Aircraft",
        	"Type Of Traveller",
        	"Seat Type",
        	"Route",
        	CAST(NULLIF("Date Flown", 'NaN') AS TIMESTAMP),
        	CAST("Seat Comfort" AS FLOAT8),
        	CAST("Cabin Staff Service" AS FLOAT8),
        	CAST("Food & Beverages" AS FLOAT8),
        	CAST("Ground Service" AS FLOAT8),
        	CAST("Inflight Entertainment" AS FLOAT8),
        	CAST("Wifi & Connectivity" AS FLOAT8),
        	CAST("Value For Money" AS FLOAT8),
        	"Recommended",
        	sentiment_label,
            CAST(sentiment_scores AS FLOAT8),
            CAST("Overall_Rating" AS FLOAT8),
            CAST(is_lost_luggage_flag AS BOOL),
            CAST(lost_luggage_cosim AS FLOAT8), 
            'From_AirlineQuality' AS created_by,
            NOW() AS created_date,
            'From_AirlineQuality' AS lastmodified_by,
            NOW() AS lastmodified_date
        FROM bronze_airline_quality_reviews AS S
        WHERE NOT EXISTS
        (
            SELECT D2."Id"
            FROM silver_airline_quality_reviews AS D2
            WHERE D2."Id" = S."Id"
        )
    """)
    rowcount = cursor.rowcount
    
    cursor.execute("""
                DROP TABLE IF EXISTS bronze_airline_quality_reviews;
                """)
    
    connection.commit()
    print(f'{rowcount} records inserted to "silver_airline_quality_reviews" successfully')


except (Exception, psycopg2.Error) as error:
    print("Error during insert operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()

52 records inserted to "silver_airline_quality_reviews" successfully
